# 특허 문서 분류

1. 특허문서의 구성
    - 기술체계(대분류, 중분류, 소분류, 핵심기술, 출원번호, 출원일자, 요약문)
    
2. 비구조화된 요약문을 구조화
    - 대문자 --> 소문자
    - 불용어 제거
    - stemming

3. training / test set
    - 출원연도를 기준
    
4. 모형 비교
    - top 20 terms
    - TSNE
    - PCA

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import time
import sys
import io

In [2]:
np.random.seed(777)

In [21]:
reload(sys)  
sys.setdefaultencoding('utf-8')

In [3]:
data = pd.read_csv(r'C:\kjpy\input\patent\data_kr.csv', encoding = 'euc-kr')

In [4]:
data

,industry,class_1,class_2,tech_core,appln_no,appln_date,appln_nat_cd,abstract
0,로봇,로봇,부품,a,102013000007871,20130124,KR,"본 발명은 내부배선을 적용한 뱀 로봇용 양단지지형 구동모듈에 관한 것으로, 특히 뱀..."
1,로봇,로봇,부품,a,101997000030000,19970630,KR,"본 발명은 유압식 롤 피치 드라이브에 관한 것으로, 각각 롤 구동요소(200)와 피..."
2,로봇,로봇,부품,a,101998000000931,19980115,KR,본 발명은 인간근육의 특징을 이용한 다자유도계 운동모듈에 관한 것이다. 본 발명에서...
3,로봇,로봇,부품,a,102013000052173,20130508,KR,수술로봇의 제어가 보다 직관적으로 이루어지도록 햅틱 피드백을 제공하는 햅틱 글로브 ...
4,로봇,로봇,부품,a,102013000071056,20130620,KR,본 발명은 초소형 로봇의 선체에 탈부착이 용이한 소나 센서 및 이를 구비한 소나 시...
5,로봇,로봇,부품,a,102014000037094,20140328,KR,"본 발명은 하지와 상지 사이에 형성되어 충격 및 진동을 최소화하고, 상지 및 하지를..."
6,로봇,로봇,부품,a,102012000094085,20120828,KR,"다이얼 방식으로 용접 전압/전류를 조정함으로써, 다양한 용접 상황에 신속하게 대처하..."
7,로봇,로봇,부품,a,102006000028701,20060330,KR,본 발명의 과제는 간단한 구조임에도 불구하고 높은 검출 정밀도를 갖는 생산성이 우수...
8,로봇,로봇,부품,a,102006000070838,20060727,KR,본 발명은 인공표식과 오도메트리를 결합한 실시간 위치산출 시스템 및 방법에 관한 것...
9,로봇,로봇,부품,a,102009000056304,20090624,KR,본발명은 접촉힘 세기 또는 압력 세기 감지하는 촉각센서를 이용하여 조도 조절 가능한...


In [5]:
data['year'] = data['appln_date'].apply(lambda x: int(x/10000))

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 9 columns):
industry        82 non-null object
class_1         82 non-null object
class_2         82 non-null object
tech_core       82 non-null object
appln_no        82 non-null int64
appln_date      82 non-null int64
appln_nat_cd    82 non-null object
abstract        82 non-null object
year            82 non-null int64
dtypes: int64(3), object(6)
memory usage: 5.8+ KB


In [15]:
abstract = data['abstract']

In [16]:
abstract

0     본 발명은 내부배선을 적용한 뱀 로봇용 양단지지형 구동모듈에 관한 것으로, 특히 뱀...
1     본 발명은 유압식 롤 피치 드라이브에 관한 것으로, 각각 롤 구동요소(200)와 피...
2     본 발명은 인간근육의 특징을 이용한 다자유도계 운동모듈에 관한 것이다. 본 발명에서...
3     수술로봇의 제어가 보다 직관적으로 이루어지도록 햅틱 피드백을 제공하는 햅틱 글로브 ...
4     본 발명은 초소형 로봇의 선체에 탈부착이 용이한 소나 센서 및 이를 구비한 소나 시...
5     본 발명은 하지와 상지 사이에 형성되어 충격 및 진동을 최소화하고, 상지 및 하지를...
6     다이얼 방식으로 용접 전압/전류를 조정함으로써, 다양한 용접 상황에 신속하게 대처하...
7     본 발명의 과제는 간단한 구조임에도 불구하고 높은 검출 정밀도를 갖는 생산성이 우수...
8     본 발명은 인공표식과 오도메트리를 결합한 실시간 위치산출 시스템 및 방법에 관한 것...
9     본발명은 접촉힘 세기 또는 압력 세기 감지하는 촉각센서를 이용하여 조도 조절 가능한...
10    본 발명은 도착시간지연 특성행렬을 이용하여, 측정된 도착시간지연 값에 대한 화자의 ...
11    본 발명은 이방성 초음파센서와 무지향성 적외선 발사장치를 구비한 이동로봇 및 그를 ...
12    본 발명은 무인 이동 물체의 비젼 시스템에서의 비젼 센서 장치와 이를 이용한 위치 ...
13    물체의 인식 및 자세 추정 방법은 스테레오 카메라를 통하여 2차원 영상을 획득하는 ...
14    본 발명은 다관절 로봇에 이용되는 소형 액추에이터 모듈에 대한 것으로서, 특히 마스...
15    본 발명은 리프팅 러그 용접 로봇 시스템에 관한 것으로, 소정 개의 러그(3)를 규...
16    본 발명의 실시 예들은, 단말 장치와 로봇 청소기를 네트워크를 통해 연결함으로써 로...
17    본 발명은 로봇의 위치를 추정하기 위한 위치결정서비스 프레임 워크 장치 및 

In [9]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Hannanum

In [10]:
kkma = Kkma()

In [17]:
abstract = abstract.apply(lambda x: kkma.nouns(x))

In [18]:
abstract

0     [발명, 내부, 내부배선, 배선, 적용, 뱀, 로봇, 양, 양단지지형, 단지, 지형...
1     [발명, 유압식, 롤, 피치, 드라이브, 각각, 구동, 구동요소, 요소, 200, ...
2     [발명, 인간, 인간근육, 근육, 특징, 이용, 자유도, 운동, 운동모듈, 모듈, ...
3     [수술, 수술로봇, 로봇, 제어, 직관적, 햅틱, 피드백, 제공, 글로브, 이, 포...
4     [발명, 초소형, 로봇, 선체, 부착, 소나, 센서, 구비, 시스템, 10, 형성,...
5     [발명, 하지, 상지, 사이, 형성, 충격, 진동, 최소화, 수동, 능동, 제어, ...
6     [다이얼, 방식, 용접, 전압, 전류, 조정, 다양, 상황, 신속, 대처, 편리함,...
7     [발명, 과제, 구조, 검출, 정밀도, 생산성, 우수, 속도, 센서, 제공, 베이스...
8     [발명, 인공, 인공표식, 표식, 오도, 오도메트리, 메트리, 결합, 실시간, 위치...
9     [발명, 접촉, 접촉힘, 힘, 세기, 압력, 감지, 촉각, 촉각센서, 센서, 이용,...
10    [발명, 도착, 도착시간지연, 시간, 지연, 특성, 특성행렬, 행렬, 이용, 측정,...
11    [발명, 이방성, 초음파, 초음파센서, 센서, 지향성, 적외선, 발사, 발사장치, ...
12    [발명, 무인, 이동, 물체, 비젼, 시스템, 센서, 장치, 이용, 위치, 추정, ...
13    [물체, 인식, 자세, 추정, 방법, 스테레오, 카메라, 2, 2차원, 차원, 영상...
14    [발명, 다관절, 로봇, 이용, 소형, 액, 액추, 추, 터, 모듈, 마스터, 레,...
15    [발명, 리프팅, 러그, 용접, 로봇, 시스템, 소정, 개의, 3, 규칙적, 배열,...
16    [발명, 실시, 예, 단말, 장치, 로봇, 청소기, 네트워크, 연결, 다양, 방식,...
17    [발명, 로봇, 위치, 추정, 위치결정서비스, 결정, 서비스, 프레임, 워크

####prepare for vectorizer

In [19]:
abstract = abstract.apply(lambda x: " ".join(x))

In [20]:
abstract

0     발명 내부 내부배선 배선 적용 뱀 로봇 양 양단지지형 단지 지형 구동 구동모듈 모듈...
1     발명 유압식 롤 피치 드라이브 각각 구동 구동요소 요소 200 300 원호 왕복 왕...
2     발명 인간 인간근육 근육 특징 이용 자유도 운동 운동모듈 모듈 로봇 등 팔 전달 동...
3     수술 수술로봇 로봇 제어 직관적 햅틱 피드백 제공 글로브 이 포함 시스템 조작자 을...
4     발명 초소형 로봇 선체 부착 소나 센서 구비 시스템 10 형성 부착부 부 30 의 ...
5     발명 하지 상지 사이 형성 충격 진동 최소화 수동 능동 제어 직렬 성구 성구동기 동...
6     다이얼 방식 용접 전압 전류 조정 다양 상황 신속 대처 편리함 도모 로봇 시스템 개...
7     발명 과제 구조 검출 정밀도 생산성 우수 속도 센서 제공 베이스 13 연장 복수 아...
8     발명 인공 인공표식 표식 오도 오도메트리 메트리 결합 실시간 위치 위치산출 산출 시...
9     발명 접촉 접촉힘 힘 세기 압력 감지 촉각 촉각센서 센서 이용 조도 조절 가능 조명...
10    발명 도착 도착시간지연 시간 지연 특성 특성행렬 행렬 이용 측정 값 화자 방위 방위...
11    발명 이방성 초음파 초음파센서 센서 지향성 적외선 발사 발사장치 장치 구비 이동 이...
12    발명 무인 이동 물체 비젼 시스템 센서 장치 이용 위치 추정 방법 본 상기 주변 빛...
13    물체 인식 자세 추정 방법 스테레오 카메라 2 2차원 차원 영상 획득 단계 3 3차...
14    발명 다관절 로봇 이용 소형 액 액추 추 터 모듈 마스터 레 레이브 이브 방식 동작...
15    발명 리프팅 러그 용접 로봇 시스템 소정 개의 3 규칙적 배열 적치 수 러그적치대 ...
16    발명 실시 예 단말 장치 로봇 청소기 네트워크 연결 다양 방식 제어 가능 본 외부 ...
17    발명 로봇 위치 추정 위치결정서비스 결정 서비스 프레임 워크 장치 동작 방법

In [21]:
vectorizer = CountVectorizer(max_features = 20)

In [22]:
##set x, y
x = vectorizer.fit_transform(abstract)

In [23]:
x = x.toarray()

In [24]:
y=LabelEncoder().fit_transform(data['tech_core'])

In [25]:
print('# x {} y {}'.format(x.shape, y.shape))

# x (82L, 20L) y (82L,)


##################################################################################################################
    ### Cross Validation
    ##################################################################################################################


In [26]:
 model = RandomForestClassifier(max_depth=5, n_jobs=-1, random_state=777)

In [27]:
x_trn = x[data[data['year'] <= 2010].index.values]
x_vld = x[data[data['year'] >= 2011].index.values]
y_trn = y[data[data['year'] <= 2010].index.values]
y_vld = y[data[data['year'] >= 2011].index.values]

In [28]:
print('# trn {} vld {}'.format(x_trn.shape, x_vld.shape))

# trn (61L, 20L) vld (21L, 20L)


In [29]:
model.fit(x_trn, y_trn)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=777,
            verbose=0, warm_start=False)

In [30]:
trn_score = accuracy_score(y_trn, model.predict(x_trn))
vld_score = accuracy_score(y_vld, model.predict(x_vld))

In [31]:
print('# TRN accuracy: {}'.format(trn_score))
print('# VLD accuracy: {}'.format(vld_score))

# TRN accuracy: 0.934426229508
# VLD accuracy: 0.619047619048


TSNE 
http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [32]:
from sklearn.manifold import TSNE

In [34]:
vectorizer_TSNE = CountVectorizer()

In [35]:
x_TSNE = vectorizer_TSNE.fit_transform(abstract)

In [36]:
x_TSNE = x_TSNE.toarray()

In [37]:
x_TSNE

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
print('# x_TSNE {} y {}'.format(x_TSNE.shape, y.shape))

# x_TSNE (82L, 1783L) y (82L,)


In [39]:
TSNE_model = TSNE(n_components=2, random_state=0)

In [40]:
x_1 = TSNE_model.fit_transform(x_TSNE)

In [41]:
print('# x_1 {} y {}'.format(x_1.shape, y.shape))

# x_1 (82L, 2L) y (82L,)


In [42]:
x_1_trn = x_1[data[data['year'] <= 2010].index.values]
x_1_vld = x_1[data[data['year'] >= 2011].index.values]
y_trn = y[data[data['year'] <= 2010].index.values]
y_vld = y[data[data['year'] >= 2011].index.values]

In [43]:
print('# trn {} vld {}'.format(x_1_trn.shape, x_1_vld.shape))

# trn (61L, 2L) vld (21L, 2L)


In [44]:
model.fit(x_1_trn, y_trn)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=777,
            verbose=0, warm_start=False)

In [45]:
trn_1_score = accuracy_score(y_trn, model.predict(x_1_trn))
vld_1_score = accuracy_score(y_vld, model.predict(x_1_vld))

In [46]:
print('# TRN_1 accuracy: {}'.format(trn_1_score))
print('# VLD_1 accuracy: {}'.format(vld_1_score))

# TRN_1 accuracy: 0.918032786885
# VLD_1 accuracy: 0.714285714286


PCA
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [47]:
from sklearn.decomposition import PCA

In [50]:
pca = PCA(n_components = 3)

In [48]:
vectorizer_PCA = CountVectorizer()
x_PCA = vectorizer_PCA.fit_transform(abstract)
x_PCA = x_PCA.toarray()

In [51]:
pca.fit(x_PCA)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [52]:
x_2 = PCA(n_components=4).fit_transform(x_PCA)

In [53]:
x_2

array([[ -1.55459364e+00,   2.45075674e+00,  -2.08650443e+00,
         -8.78621579e-01],
       [ -1.32753127e+00,   1.30022035e+00,  -1.08353253e+00,
         -1.02526088e+00],
       [ -1.01079590e+00,   1.93508641e+00,  -1.06173358e+00,
         -1.21235377e+00],
       [ -1.96449864e-01,   2.30552367e-02,  -6.96796110e-01,
          2.19533571e-01],
       [  4.40792169e-01,   6.50260790e-01,  -8.75512024e-01,
         -5.57128716e-01],
       [ -7.30840788e-01,   1.96981810e-01,  -3.87773015e-01,
         -7.18327640e-02],
       [  2.15021571e-01,   2.83085134e-02,   1.20863454e-01,
          3.48424823e-01],
       [ -3.13417966e-01,  -3.56719561e-01,  -2.89449606e-01,
         -1.13796266e+00],
       [  6.50442746e-01,  -1.18308454e+00,   3.77847740e-02,
         -2.23472734e-02],
       [ -2.37457189e-01,  -3.78462725e-02,  -3.03451766e-01,
         -1.99004724e-02],
       [  1.83145871e-01,  -1.08498598e+00,   1.33920147e-01,
         -2.37488278e-01],
       [ -2.32386223e

In [54]:
print('# x_2 {} y {}'.format(x_2.shape, y.shape))

# x_2 (82L, 4L) y (82L,)


In [55]:
x_2_trn = x_2[data[data['year'] <= 2010].index.values]
x_2_vld = x_2[data[data['year'] >= 2011].index.values]
y_trn = y[data[data['year'] <= 2010].index.values]
y_vld = y[data[data['year'] >= 2011].index.values]

In [56]:
print('# trn {} vld {}'.format(x_2_trn.shape, x_2_vld.shape))

# trn (61L, 4L) vld (21L, 4L)


In [57]:
model.fit(x_2_trn, y_trn)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=777,
            verbose=0, warm_start=False)

In [58]:
trn_2_score = accuracy_score(y_trn, model.predict(x_2_trn))
vld_2_score = accuracy_score(y_vld,  )

In [59]:
print('# TRN_2 accuracy: {}'.format(trn_2_score))
print('# VLD_2 accuracy: {}'.format(vld_2_score))

# TRN_2 accuracy: 0.983606557377
# VLD_2 accuracy: 0.714285714286


In [60]:
model.predict(x_2_vld) 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0], dtype=int64)